# Outline

0. Instructions
1. Intro to topic modeling reading (~5 minutes)
2. Setup
3. Preview
4. Load data
5. First preprocessing experiment: Implement a function for the first preprocessing experiment
6. Create your first topic model
7. Qualitative observations of first topic model.
8. Quantitative observations of first topic model.
9. Second preprocessing experiment: Implement a different preprocessing function for the second preprocessing experiment and run topic modeling pipeline on new preprocessed data.
11. Parameter Experiment
12. Conclusions

# Instructions

1. This notebook will guide you through each step of the project. 
2. Throughout the project, you will record your experiments and observations. We have created a slide deck template for you that you can download and edit in Google Slides or Microsoft Power Point. The template is at **`Experiment-Report-Templates/2-Topic-Modeling-Template.pptx`**.
3. We are happy to help with *anything*. Ask away :) 

# Introduction to Topic Modeling Reading

![TM](https://raw.githubusercontent.com/alahnala/AI4All2020-Michigan-NLP/master/slides/tm-0.png)

![TM](https://raw.githubusercontent.com/alahnala/AI4All2020-Michigan-NLP/master/slides/tm-1.png)
![TM](https://raw.githubusercontent.com/alahnala/AI4All2020-Michigan-NLP/master/slides/tm-2.png)
![TM](https://raw.githubusercontent.com/alahnala/AI4All2020-Michigan-NLP/master/slides/tm-3.png)
![TM](https://raw.githubusercontent.com/alahnala/AI4All2020-Michigan-NLP/master/slides/tm-4.png)

# Setup

Run the cell below to import functions written by staff and functions from libraries. You will not need to worry about the details these, but if you are curious to see what they look like, checkout the file `tm_helpers.py`. If you want to learn more about how these functions or have questions, let us know! :)

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/My Drive/My-AI4All-NLP-Project'
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.stem.snowball import PorterStemmer
from utils.nlp_basics import *
from utils.syllable import *

import utils.tm_helpers as helpers

# general
from tqdm import tqdm
import os
import regex as re

# preprocess functions
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])


# topic modeling packages
import gensim
from gensim import models, corpora
from gensim.models.coherencemodel import CoherenceModel

# used to visualize the topic model
import pyLDAvis.gensim
import pyLDAvis

print('Done')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allielahnala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Done


# Preview

By the end of this project, you will have experimented with preprocessing steps, and generate an interactive visual to explore the topics. Run the cell below to get a preview of the end goal.

In [2]:
helpers.show_model()

Generating visual, this will take a few moments...


/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x        y  topics  cluster       Freq
topic                                               
2      0.081952  0.14081       1        1  35.040768
0     -0.177240 -0.00689       2        1  33.574291
1      0.095287 -0.13392       3        1  31.384935, topic_info=     Category       Freq              Term      Total  loglift  logprob
term                                                                   
453   Default  85.000000              cake  85.000000  30.0000  30.0000
6     Default  48.000000          computer  48.000000  29.0000  29.0000
1     Default  32.000000  computer_science  32.000000  28.0000  28.0000
1045  Default  22.000000              verb  22.000000  27.0000  27.0000
494   Default  25.000000           century  25.000000  26.0000  26.0000
825   Default  14.000000             vowel  14.000000  25.0000  25.0000
12    Default  20.000000            system  20.000000  24.0000  24.0000
1172  Default  13.000000       alternation  13.000000  23.0000  23.0000
0     Default  14.000000       computation  14.000000  22.0000  22.0000
821   Default  13.000000          syllable  13.000000  21.0000  21.0000
34    Default  13.000000       programming  13.000000  20.0000  20.0000
59    Default  13.000000       development  13.000000  19.0000  19.0000
3     Default  12.000000             study  12.000000  18.0000  18.0000
13    Default  12.000000            theory  12.000000  17.0000  17.0000
454   Default  13.000000             flour  13.000000  16.0000  16.0000
278   Default  37.000000              form  37.000000  15.0000  15.0000
8     Default  12.000000            design  12.000000  14.0000  14.0000
1140  Default  11.000000             class  11.000000  13.0000  13.0000
2     Default  11.000000       information  11.000000  12.0000  12.0000
23    Default  11.000000        discipline  11.000000  11.0000  11.0000
1089  Default  13.000000              noun  13.000000  10.0000  10.0000
1156  Default  10.000000         adjective  10.000000   9.0000   9.0000
24    Default  13.000000             field  13.000000   8.0000   8.0000
1141  Default  10.000000        declension  10.000000   7.0000   7.0000
11    Default  10.000000          software  10.000000   6.0000   6.0000
97    Default  10.000000           science  10.000000   5.0000   5.0000
15    Default  10.000000         computing  10.000000   4.0000   4.0000
457   Default  10.000000             sugar  10.000000   3.0000   3.0000
1139  Default   8.000000          singular   8.000000   2.0000   2.0000
1280  Default   8.000000        participle   8.000000   1.0000   1.0000
...       ...        ...               ...        ...      ...      ...
1059   Topic3   6.941285              loss   7.469290   1.0855  -4.9950
1047   Topic3   6.541629         diphthong   7.070148   1.0811  -5.0543
1180   Topic3   6.238444                je   6.765184   1.0778  -5.1017
1155   Topic3   6.237119              stem   6.765402   1.0775  -5.1020
1024   Topic3   5.293692             nasal   5.819977   1.0641  -5.2660
424    Topic3   4.731692            result   5.259983   1.0530  -5.3782
995    Topic3   4.358325                 t   4.885259   1.0447  -5.4604
993    Topic3   3.728317         consonant   4.255137   1.0267  -5.6165
1190   Topic3   3.654912       subjunctive   4.181459   1.0243  -5.6364
1142   Topic3   3.497585               -er   4.023951   1.0187  -5.6804
1089   Topic3  11.762541              noun  13.690912   1.0070  -4.4676
1133   Topic3   3.179449            neuter   3.709897   1.0045  -5.7758
816    Topic3   3.024989                 e   3.551349   0.9984  -5.8256
839    Topic3   3.024962                 l   3.551353   0.9984  -5.8256
952    Topic3   3.023741             music   3.551555   0.9980  -5.8260
757    Topic3   2.998004           cluster   3.525030   0.9969  -5.8345
1246   Topic3   2.794014            desjun   3.320170   0.9863  -5.9050
1205   Topic3   2.794011             value   3.320168   0.9863  -5.9050
1286   Topic3   2.79

# Data

## Load data

Run the cell below to load the data that we will use for topic modeling. The data is saved into a *list* data structure called `data`.

In [3]:
data = helpers.load_data()

**To get an idea of what `data` looks like, run the cell below to see what the first  looks like.**

In [4]:
print("Data length:", len(data), "First 10 items in data:\n")

# data is a list, data[:10] is the first ten items of that list
for i, item in enumerate(data[:10]):
    print("{}:".format(i), item)

Data length: 494 First 10 items in data:

0: Computer science is the study of computation and information.

1: Computer science deals with theory of computation, algorithms, computational problems and the design of computer systems hardware, software and applications.

2: Computer science addresses both human-made and natural information processes, such as communication, control, perception, learning and intelligence especially in human-made computing systems and machines.

3: According to Peter Denning, the fundamental question underlying computer science is, What can be automated?Its fields can be divided into theoretical and practical disciplines.

4: Computational complexity theory is highly abstract, while computer graphics and computational geometry emphasizes real-world applications.

5: Algorithmics is called the heart of computer science.

6: Programming language theory considers approaches to the description of computational processes, while software engineering involves the 

# Preprocessing Experiment 1

## Write the function `preprocess_line`.
1. At this point, *preprocessing* is the main piece to experiment with and observe your implementation decisions on topic modeling. Using at least one new thing that you learned in 1-Intro-to-NLP, write the function `preprocess_line` which takes in a string of text saved in the input variable `line`, so that the function returns a list of tokens called `preprocessed_line`. 
2. At the bottom of the cell, we call `preprocess_line` on `test_string` so you can quickly observe how your function is working. `test_string` is initially set to be the first string from `data`, but you can change `test_string` to another string from `data` or your own string to get a fuller idea of the effects of your function.
3. Take a second to record your decisions in the slide titled **Preprocessing Experiment 1**.


In [5]:
def preprocess_line(line):
    '''
    Fill in this function. Refer to 1-Intro-to-NLP for preprocessing ideas.
    '''
    preprocessed_line = line.split()

    
    return preprocessed_line

test_string = data[0]
print(preprocess_line(test_string))

['Computer', 'science', 'is', 'the', 'study', 'of', 'computation', 'and', 'information.']


## Run `preprocess` on all of the data.
1. Run the cell below to `preprocess` all of the strings in `data` and save to `preprocessed_data` 
2. The cell will output a preview our `preprocessed_data`. How does is look different than our earlier preview? Do you have a bug or does it look how you want it to look? 
3. Take a second to record your observations in the slide deck, in the slide titled **Preprocessing Output Observations**.

In [6]:
def preprocess(data):
    preprocessed_data = []
    for line in tqdm(data):
        preprocessed_line = preprocess_line(line)
        preprocessed_data.append(preprocessed_line)
    return preprocessed_data
preprocessed_data = preprocess(data)

# data is a list, data[:10] is the first ten items of that list
for i, item in enumerate(preprocessed_data[:10]):
    print("{}:".format(i), item)

100%|██████████| 494/494 [00:00<00:00, 143857.96it/s]

0: ['Computer', 'science', 'is', 'the', 'study', 'of', 'computation', 'and', 'information.']
1: ['Computer', 'science', 'deals', 'with', 'theory', 'of', 'computation,', 'algorithms,', 'computational', 'problems', 'and', 'the', 'design', 'of', 'computer', 'systems', 'hardware,', 'software', 'and', 'applications.']
2: ['Computer', 'science', 'addresses', 'both', 'human-made', 'and', 'natural', 'information', 'processes,', 'such', 'as', 'communication,', 'control,', 'perception,', 'learning', 'and', 'intelligence', 'especially', 'in', 'human-made', 'computing', 'systems', 'and', 'machines.']
3: ['According', 'to', 'Peter', 'Denning,', 'the', 'fundamental', 'question', 'underlying', 'computer', 'science', 'is,', 'What', 'can', 'be', 'automated?Its', 'fields', 'can', 'be', 'divided', 'into', 'theoretical', 'and', 'practical', 'disciplines.']
4: ['Computational', 'complexity', 'theory', 'is', 'highly', 'abstract,', 'while', 'computer', 'graphics', 'and', 'computational', 'geometry', 'emphasi

## Final data preparation for gensim topic modeling

1. In this step, we use functions from the `gensim` module to create `dictionary` and `corpus` in the format that the topic modeling function requires.
2. `dictionary`: This is our *set of words* after our preprocessing, which we will use for creating different probability distributions. This *set of words* is often referred to as the *vocabulary* in NLP terminology.
3. `corpus`: Words are typically converted into a numerical representation before using them in a model. To demonstrate this, the cell will print each word for the first item in your preprocessed data next to its numerical encoding. Why is the word **information** represented as **(3, 1)**?

In [7]:
print("Creating dictionary and corpus instances for gensim...", end='')

dictionary = corpora.Dictionary(preprocessed_data)
corpus = [dictionary.doc2bow(x) for x in preprocessed_data]

print("complete.\n")

print(dictionary, '\n')


for original, encoded in zip(preprocessed_data[0], corpus[0]):
    print("Original: {}".format(original), "--->  Encoded: {}".format(encoded))

Creating dictionary and corpus instances for gensim...complete.

Dictionary(4371 unique tokens: ['Computer', 'and', 'computation', 'information.', 'is']...) 

Original: Computer --->  Encoded: (0, 1)
Original: science --->  Encoded: (1, 1)
Original: is --->  Encoded: (2, 1)
Original: the --->  Encoded: (3, 1)
Original: study --->  Encoded: (4, 1)
Original: of --->  Encoded: (5, 1)
Original: computation --->  Encoded: (6, 1)
Original: and --->  Encoded: (7, 1)
Original: information. --->  Encoded: (8, 1)


# Create Topic Model


1. Run the cell below to create a topic model for our data, and the topic model will be saved in the variable `lda_model`. For now, do not worry about the parameters, we will experiment with those later.

In [8]:
'''
The following lines are used to setup the model's parameters. 
'''
UPDATE_EVERY = 10
CHUNKSIZE = 10
PASSES = 10
topic_model_settings = [{'num-topics':15, 'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':5,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':10,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}]
setting = topic_model_settings[0]
NUM_TOPICS = setting['num-topics']


'''
Here, we plug in our parameters and data into models.LdaModel to train the topic model. The topic model will be saved in the variable "lda_model".
'''
print("Training topic model of {} topics (this might take a moment)...".format(NUM_TOPICS), end='')
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, random_state = setting['parameters']['random_state'], update_every = setting['parameters']['update_every'], chunksize = setting['parameters']['chunksize'], passes = setting['parameters']['passes'], alpha = setting['parameters']['alpha'], per_word_topics = setting['parameters']['per_word_topics'])
print("complete.")


Training topic model of 15 topics (this might take a moment)...complete.


# Qualitative Observations

We have created a topic model and saved it in `lda_model`. But what does this mean? Let's learn more as we perform some qualitative observations. Later, we will discuss two quantitative metrics, but a lot of times, topic models are decided from manual observation.

## Topic Terms
1. Run the cell below to observe the topic terms.  
2. By creating the topic model, we have created a probability distribution over our set of words (`dictionary` or the vocabulary) for 15 topics. We have created a helper function called `show_topic_terms` that will extract the ten words with the highest probabilities for fitting into each topic. 
3. The columns of `topic_terms` are the top ten words of each topic (Wn) and Wn's probability of belonging to each topic (Wn Pr). Each row is a topic.
4. What do you think of these topics? Can you make sense of clear topic divisions between the 15 topics, or, would you be able to give a name to each topic? How could you modify the `preprocess_line` function to possibly improve the topic model to get a clearer division between topics?
5. Before you make any changes or proceed, make notes of your observations in the slide **Topic Model Qualitative Observations**. You can include a screenshot of part of this table, paste the table values into the slides.

In [9]:
topic_terms = helpers.show_topic_terms(lda_model, NUM_TOPICS)
topic_terms

,W1,W1 Pr,W2,W2 Pr,W3,W3 Pr,W4,W4 Pr,W5,W5 Pr,W6,W6 Pr,W7,W7 Pr,W8,W8 Pr,W9,W9 Pr,W10,W10 Pr
Topic,,,,,,,,,,,,,,,,,,,,
0,several,0.017,"this,",0.011,alternative,0.007,methods,0.006,commonly,0.005,Gottfried,0.005,algorithms,0.005,structures,0.005,Data,0.005,computational,0.003
1,clear,0.013,just,0.005,general.,0.003,"calculations,",0.003,broadened,0.003,field,0.002,could,0.001,As,0.001,became,0.000,mathematical,0.000
2,the,0.098,of,0.065,and,0.060,in,0.050,a,0.034,to,0.030,is,0.022,as,0.020,with,0.018,was,0.015
3,de,0.058,only,0.039,when,0.029,his,0.014,out,0.010,although,0.009,create,0.009,earlier,0.008,means,0.007,since,0.006
4,French,0.201,Middle,0.024,became,0.020,including,0.010,system,0.010,distinct,0.008,especially,0.007,among,0.006,As,0.005,could,0.005
5,language,0.036,computer,0.027,It,0.016,languages,0.015,Computer,0.013,programming,0.013,of,0.012,and,0.012,complex,0.010,science,0.009
6,An,0.026,before,0.023,since,0.016,even,0.014,existed,0.009,performing,0.005,field,0.005,proposed,0.003,data,0.003,now,0.003
7,types,0.020,generally,0.017,Charles,0.013,design,0.012,Both,0.009,across,0.009,Information,0.008,gave,0.008,idea,0.007,Difference,0.006
8,important,0.022,no,0.018,database,0.009,distributed,0.009,about,0.008,parallel,0.007,states,0.007,network,0.007,"engineering,",0.007,"systems,",0.006


## Topic Model Visualization

1. Run the cell below to generate an interactive visual to explore the topic model.
2. Add observations to the **Topic Model Qualitative Observations** slide.

In [10]:
helpers.show_model(lda_model, corpus, dictionary)

Generating visual, this will take a few moments...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.446768  0.009418       1        1  84.542358
4     -0.022582  0.167905       2        1   3.482362
5      0.042346 -0.087548       3        1   3.086567
3     -0.043488 -0.016092       4        1   2.625262
12    -0.029964 -0.022031       5        1   1.572725
7     -0.039839 -0.010991       6        1   1.114673
8     -0.027960 -0.013384       7        1   0.901680
11    -0.037615 -0.009640       8        1   0.729298
6     -0.044029 -0.006602       9        1   0.597990
10    -0.044709 -0.005079      10        1   0.514862
0     -0.041176 -0.005843      11        1   0.353066
14    -0.041035  0.002764      12        1   0.231603
13    -0.039393 -0.000771      13        1   0.115522
1     -0.039399 -0.001104      14        1   0.108335
9     -0.037926 -0.001003      15        1   0.023686, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
244   Default   85.000000         French   85.000000  30.0000  30.0000
13    Default   79.000000       computer   79.000000  29.0000  29.0000
1     Default  621.000000            and  621.000000  28.0000  28.0000
0     Default   49.000000       Computer   49.000000  27.0000  27.0000
5     Default  679.000000             of  679.000000  26.0000  26.0000
6     Default   43.000000        science   43.000000  25.0000  25.0000
180   Default   19.000000             de   19.000000  24.0000  24.0000
4     Default  230.000000             is  230.000000  23.0000  23.0000
328   Default   24.000000           more   24.000000  22.0000  22.0000
38    Default   57.000000           such   57.000000  21.0000  21.0000
77    Default   13.000000       language   13.000000  20.0000  20.0000
150   Default   48.000000          first   48.000000  19.0000  19.0000
288   Default   12.000000           only   12.000000  18.0000  18.0000
222   Default   35.000000           than   35.000000  17.0000  17.0000
347   Default  112.000000           that  112.000000  16.0000  16.0000
132   Default  111.000000            for  111.000000  15.0000  15.0000
356   Default   16.000000     University   16.000000  14.0000  14.0000
90    Default   19.000000      computers   19.000000  13.0000  13.0000
12    Default   18.000000  computational   18.000000  12.0000  12.0000
193   Default   31.000000           used   31.000000  11.0000  11.0000
216   Default    9.000000            had    9.000000  10.0000  10.0000
194   Default    9.000000           when    9.000000   9.0000   9.0000
30    Default  518.000000             in  518.000000   8.0000   8.0000
240   Default  160.000000            was  160.000000   7.0000   7.0000
2408  Default   10.000000         Middle   10.000000   6.0000   6.0000
234   Default   63.000000             it   63.000000   5.0000   5.0000
218   Default   22.000000           many   22.000000   4.0000   4.0000
656   Default   12.000000           data   12.000000   3.0000   3.0000
26    Default   17.000000      computing   17.000000   2.0000   2.0000
18    Default   20.000000       software   20.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
4370  Topic15    0.000661    L'Angelier.    0.391960   1.9625  -8.3827
1458  Topic15    0.000661        Boehm's    0.391883   1.9627  -8.3827
1452  Topic15    0.000661            set    0.393002   1.9598  -8.3827
1453  Topic15    0.000661  such-and-such    0.392363   1.9614  -8.3827
1454  Topic15    0.000661          that;    0.392363   1.9614  -8.3827
1455  Topic15    0.000661           then    0.393186   1.9593  -8.3827
1456  Topic15    0.000661          this.    0.391864   1.9627  -8.3827
1457  Topic15    0.000661           ways    0.393209   1.9593  -8.3827
1459  Topic15    0.000661           Note    0.391883   1.9627  -8.3827
1501  Topic15    0.000661      operates.    0.391889   1.

# Quantitative Observations

1. We will compute *perplexity* and *coherence,* which are two quantitative metrics for evaluating topic models. Better scores do not necessarily mean better models in topic modeling. 
2. Add the scores to the **Topic Model 1 Quantitative Observations** slide.


## Perplexity

"In information theory, perplexity is a measurement of how well a probability distribution or probability model predicts a sample ([Wikipedia](https://en.wikipedia.org/wiki/Perplexity))."

Intuition: the dictionary definition of perplexed is "Completely baffled; very puzzled [Dictionary Reference](https://www.lexico.com/en/definition/perplexed)." The more perplexed you are, the more puzzled you are. Use this to remember the intuition behind the *perplexity* metric. We aim for a model that less perplexed by new data, so a smaller perplexity is usually the goal.


In [11]:
'''
Measure quality of topic models with perplexity
'''
print("Measuring model perplexity...",end="")
ppl = lda_model.log_perplexity(corpus)
print('complete. Perplexity:', ppl, '\n')

Measuring model perplexity...complete. Perplexity: -12.83001906315796 



## Coherence

The coherence metric captures the "degree of semantic similarity between high scoring words in the topic ([Towards Data Science](https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0))."

The idea behind the *coherence* metric aligns with your experience observing your topic terms. Did you observe a very clear theme in each topic or was there a lot of overlapping themes between the topics? Clear themes are analogous to higher coherence scores, whereas overlapping themes are analogous to distinct topics.

In [12]:
'''
Measure quality of topic models with coherence
'''
print("Measuring model coherence...",end="")
coherence_model_lda = CoherenceModel(model=lda_model, texts = preprocessed_data, corpus=corpus, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('complete. Coherence Score:', coherence_lda, '\n') 

Measuring model coherence...complete. Coherence Score: 0.4411584153231868 



# Preprocessing Experiment 2

## Write the function `preprocess_line_2`.
1. As you did with `preprocessed_line`, implement the function `preprocessed_line_2`. Try to base your new decisions on your previous observations.  
2. Test your function on a `test_string`.
3. Take a second to record your decisions in the slide titled **Preprocessing Experiment 2**.


In [13]:
def preprocess_line_2(line):
    '''
    Fill in this function. Refer to 1-Intro-to-NLP for preprocessing ideas.
    '''
    preprocessed_line = line.split()
    

    
    return preprocessed_line

test_string = data[0]
print(preprocess_line_2(test_string))

['Computer', 'science', 'is', 'the', 'study', 'of', 'computation', 'and', 'information.']


# Run Topic Modeling pipeline for preprocessing experiment 2

1. Run the following cell to create a new topic model and output the quantitative metrics. Add your observations to the **Topic Model 2 Quantitative Observations** slide.
2. The next cell will output the topic terms. Add your observations to the **Topic Model 2 Qualitative Observations** slide.
3. The third cell will output the interactive visual. Add your observations to the **Topic Model 2 Qualitative Observations** slide.

In [14]:
'''
1. Preprocess all data
'''
def preprocess_2(data):
    preprocessed_data = []
    for line in tqdm(data):
        preprocessed_line = preprocess_line_2(line)
        preprocessed_data.append(preprocessed_line)
    return preprocessed_data
preprocessed_data = preprocess_2(data)

for i, item in enumerate(preprocessed_data[:10]):
    print("{}:".format(i), item)
    
    
'''
2. Create corpus and dictionary
'''   
print("Creating dictionary and corpus instances for gensim...", end='')
dictionary = corpora.Dictionary(preprocessed_data)
corpus = [dictionary.doc2bow(x) for x in preprocessed_data]
print("complete.\n")

'''
3. Create topic model
'''
print("Training topic model of {} topics (this might take a moment)...".format(NUM_TOPICS), end='')
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, random_state = setting['parameters']['random_state'], update_every = setting['parameters']['update_every'], chunksize = setting['parameters']['chunksize'], passes = setting['parameters']['passes'], alpha = setting['parameters']['alpha'], per_word_topics = setting['parameters']['per_word_topics'])
print("complete.")

'''
4. Measure quality of topic model with perplexity
'''
print("Measuring model perplexity...",end="")
ppl = lda_model.log_perplexity(corpus)
print('complete. Perplexity:', ppl, '\n')

'''
5. Measure quality of topic models with coherence
'''
print("Measuring model coherence...",end="")
coherence_model_lda = CoherenceModel(model=lda_model, texts = preprocessed_data, corpus=corpus, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('complete. Coherence Score:', coherence_lda, '\n') 

100%|██████████| 494/494 [00:00<00:00, 192510.10it/s]

0: ['Computer', 'science', 'is', 'the', 'study', 'of', 'computation', 'and', 'information.']
1: ['Computer', 'science', 'deals', 'with', 'theory', 'of', 'computation,', 'algorithms,', 'computational', 'problems', 'and', 'the', 'design', 'of', 'computer', 'systems', 'hardware,', 'software', 'and', 'applications.']
2: ['Computer', 'science', 'addresses', 'both', 'human-made', 'and', 'natural', 'information', 'processes,', 'such', 'as', 'communication,', 'control,', 'perception,', 'learning', 'and', 'intelligence', 'especially', 'in', 'human-made', 'computing', 'systems', 'and', 'machines.']
3: ['According', 'to', 'Peter', 'Denning,', 'the', 'fundamental', 'question', 'underlying', 'computer', 'science', 'is,', 'What', 'can', 'be', 'automated?Its', 'fields', 'can', 'be', 'divided', 'into', 'theoretical', 'and', 'practical', 'disciplines.']
4: ['Computational', 'complexity', 'theory', 'is', 'highly', 'abstract,', 'while', 'computer', 'graphics', 'and', 'computational', 'geometry', 'emphasi

complete.
Measuring model perplexity...complete. Perplexity: -12.830018599298175 

Measuring model coherence...complete. Coherence Score: 0.4411584153231868 



In [15]:
'''
6. Print topic terms
'''
topic_terms = helpers.show_topic_terms(lda_model, NUM_TOPICS)
topic_terms

,W1,W1 Pr,W2,W2 Pr,W3,W3 Pr,W4,W4 Pr,W5,W5 Pr,W6,W6 Pr,W7,W7 Pr,W8,W8 Pr,W9,W9 Pr,W10,W10 Pr
Topic,,,,,,,,,,,,,,,,,,,,
0,several,0.017,"this,",0.011,alternative,0.007,methods,0.006,commonly,0.005,Gottfried,0.005,algorithms,0.005,structures,0.005,Data,0.005,computational,0.003
1,clear,0.013,just,0.005,general.,0.003,"calculations,",0.003,broadened,0.003,field,0.002,could,0.001,As,0.001,became,0.000,mathematical,0.000
2,the,0.098,of,0.065,and,0.060,in,0.050,a,0.034,to,0.030,is,0.022,as,0.020,with,0.018,was,0.015
3,de,0.058,only,0.039,when,0.029,his,0.014,out,0.010,although,0.009,create,0.009,earlier,0.008,means,0.007,since,0.006
4,French,0.201,Middle,0.024,became,0.020,including,0.010,system,0.010,distinct,0.008,especially,0.007,among,0.006,As,0.005,could,0.005
5,language,0.036,computer,0.027,It,0.016,languages,0.015,Computer,0.013,programming,0.013,of,0.012,and,0.012,complex,0.010,science,0.009
6,An,0.026,before,0.023,since,0.016,even,0.014,existed,0.009,performing,0.005,field,0.005,proposed,0.003,data,0.003,now,0.003
7,types,0.020,generally,0.017,Charles,0.013,design,0.012,Both,0.009,across,0.009,Information,0.008,gave,0.008,idea,0.007,Difference,0.006
8,important,0.022,no,0.018,database,0.009,distributed,0.009,about,0.008,parallel,0.007,states,0.007,network,0.007,"engineering,",0.007,"systems,",0.006


In [16]:
'''
7. Generate interactive visual
'''
helpers.show_model(lda_model, corpus, dictionary)

Generating visual, this will take a few moments...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.446768  0.009418       1        1  84.542297
4     -0.022582  0.167905       2        1   3.482391
5      0.042346 -0.087548       3        1   3.086651
3     -0.043488 -0.016092       4        1   2.625241
12    -0.029964 -0.022031       5        1   1.572702
7     -0.039839 -0.010991       6        1   1.114667
8     -0.027960 -0.013384       7        1   0.901683
11    -0.037615 -0.009640       8        1   0.729298
6     -0.044029 -0.006602       9        1   0.597988
10    -0.044709 -0.005079      10        1   0.514862
0     -0.041176 -0.005843      11        1   0.353066
14    -0.041035  0.002764      12        1   0.231599
13    -0.039393 -0.000771      13        1   0.115523
1     -0.039399 -0.001104      14        1   0.108340
9     -0.037926 -0.001003      15        1   0.023686, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
244   Default   85.000000         French   85.000000  30.0000  30.0000
13    Default   79.000000       computer   79.000000  29.0000  29.0000
1     Default  621.000000            and  621.000000  28.0000  28.0000
0     Default   49.000000       Computer   49.000000  27.0000  27.0000
5     Default  679.000000             of  679.000000  26.0000  26.0000
6     Default   43.000000        science   43.000000  25.0000  25.0000
180   Default   19.000000             de   19.000000  24.0000  24.0000
4     Default  230.000000             is  230.000000  23.0000  23.0000
328   Default   24.000000           more   24.000000  22.0000  22.0000
38    Default   57.000000           such   57.000000  21.0000  21.0000
77    Default   13.000000       language   13.000000  20.0000  20.0000
150   Default   48.000000          first   48.000000  19.0000  19.0000
288   Default   12.000000           only   12.000000  18.0000  18.0000
222   Default   35.000000           than   35.000000  17.0000  17.0000
347   Default  112.000000           that  112.000000  16.0000  16.0000
132   Default  111.000000            for  111.000000  15.0000  15.0000
356   Default   16.000000     University   16.000000  14.0000  14.0000
90    Default   19.000000      computers   19.000000  13.0000  13.0000
12    Default   18.000000  computational   18.000000  12.0000  12.0000
193   Default   31.000000           used   31.000000  11.0000  11.0000
216   Default    9.000000            had    9.000000  10.0000  10.0000
194   Default    9.000000           when    9.000000   9.0000   9.0000
30    Default  518.000000             in  518.000000   8.0000   8.0000
240   Default  160.000000            was  160.000000   7.0000   7.0000
2408  Default   10.000000         Middle   10.000000   6.0000   6.0000
234   Default   63.000000             it   63.000000   5.0000   5.0000
218   Default   22.000000           many   22.000000   4.0000   4.0000
656   Default   12.000000           data   12.000000   3.0000   3.0000
26    Default   17.000000      computing   17.000000   2.0000   2.0000
18    Default   20.000000       software   20.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
4370  Topic15    0.000661    L'Angelier.    0.391961   1.9625  -8.3827
1458  Topic15    0.000661        Boehm's    0.391883   1.9627  -8.3827
1452  Topic15    0.000661            set    0.393003   1.9598  -8.3827
1453  Topic15    0.000661  such-and-such    0.392364   1.9614  -8.3827
1454  Topic15    0.000661          that;    0.392364   1.9614  -8.3827
1455  Topic15    0.000661           then    0.393187   1.9593  -8.3827
1456  Topic15    0.000661          this.    0.391865   1.9627  -8.3827
1457  Topic15    0.000661           ways    0.393210   1.9593  -8.3827
1459  Topic15    0.000661           Note    0.391883   1.9627  -8.3827
1501  Topic15    0.000661      operates.    0.391890   1.

# Parameter Experiment

1. Another impact on the quality of the topics are the different parameters.
2. For some, it is fun to learn by experience and just trying different values and making observations about the effect. Others may prefer to look at documentation or [tutorials](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/) for guidance. Reading documentation for code and libraries can be another skill to work on. Try taking a look at the [Gensim LDA Documentation](https://radimrehurek.com/gensim/models/ldamodel.html) and/or [a tutorial](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/) to see what you can learn about the different parameters.
3. Try experimenting with changing at least one parameter in the cell below, and report what parameters you are experimenting with in the **Topic Model Parameter Experiment** slide. (Hint: a good start is NUM_TOPICS)



In [17]:
UPDATE_EVERY = 10
CHUNKSIZE = 10
PASSES = 10
topic_model_settings = [{'num-topics':15, 'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':5,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}, 
                        {'num-topics':10,'parameters':{'random_state':100, 'update_every':UPDATE_EVERY, 'chunksize':CHUNKSIZE, 'passes':PASSES, 'alpha':'auto', 'per_word_topics':False}}]
setting = topic_model_settings[0]
NUM_TOPICS = setting['num-topics']



## Create Topic Model with New Parameters

Run the cell to create your new topic model.

In [18]:
print("Training topic model (this will take a moment)...", end='')
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, random_state = setting['parameters']['random_state'], update_every = setting['parameters']['update_every'], chunksize = setting['parameters']['chunksize'], passes = setting['parameters']['passes'], alpha = setting['parameters']['alpha'], per_word_topics = setting['parameters']['per_word_topics'])
print("complete.")


Training topic model (this will take a moment)...complete.


## Quantitative Metrics

Report the scores in the **Parameter Experiment Quantitative Observations** slides.

In [19]:
'''
Measure quality of topic model with perplexity
'''
print("Measuring model perplexity...",end="")
ppl = lda_model.log_perplexity(corpus)
print('complete. Perplexity:', ppl, '\n')

'''
Measure quality of topic models with coherence
'''
print("Measuring model coherence...",end="")
coherence_model_lda = CoherenceModel(model=lda_model, texts = preprocessed_data, corpus=corpus, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('complete. Coherence Score:', coherence_lda, '\n') 

Measuring model perplexity...complete. Perplexity: -12.830018599298175 

Measuring model coherence...complete. Coherence Score: 0.4411584153231868 



## Qualitative Observations

The columns of `topic_terms` are the top ten words of each topic (Wn) and Wn's probability of belonging to each topic (Wn Pr). 

Report your observations in the **Parameter Experiment Qualitative Observations** slides.

In [20]:
topic_terms = helpers.show_topic_terms(lda_model, NUM_TOPICS)
topic_terms

,W1,W1 Pr,W2,W2 Pr,W3,W3 Pr,W4,W4 Pr,W5,W5 Pr,W6,W6 Pr,W7,W7 Pr,W8,W8 Pr,W9,W9 Pr,W10,W10 Pr
Topic,,,,,,,,,,,,,,,,,,,,
0,several,0.017,"this,",0.011,alternative,0.007,methods,0.006,commonly,0.005,Gottfried,0.005,algorithms,0.005,structures,0.005,Data,0.005,computational,0.003
1,clear,0.013,just,0.005,general.,0.003,"calculations,",0.003,broadened,0.003,field,0.002,could,0.001,As,0.001,became,0.000,mathematical,0.000
2,the,0.098,of,0.065,and,0.060,in,0.050,a,0.034,to,0.030,is,0.022,as,0.020,with,0.018,was,0.015
3,de,0.058,only,0.039,when,0.029,his,0.014,out,0.010,although,0.009,create,0.009,earlier,0.008,means,0.007,since,0.006
4,French,0.201,Middle,0.024,became,0.020,including,0.010,system,0.010,distinct,0.008,especially,0.007,among,0.006,As,0.005,could,0.005
5,language,0.036,computer,0.027,It,0.016,languages,0.015,Computer,0.013,programming,0.013,of,0.012,and,0.012,complex,0.010,science,0.009
6,An,0.026,before,0.023,since,0.016,even,0.014,existed,0.009,performing,0.005,field,0.005,proposed,0.003,data,0.003,now,0.003
7,types,0.020,generally,0.017,Charles,0.013,design,0.012,Both,0.009,across,0.009,Information,0.008,gave,0.008,idea,0.007,Difference,0.006
8,important,0.022,no,0.018,database,0.009,distributed,0.009,about,0.008,parallel,0.007,states,0.007,network,0.007,"engineering,",0.007,"systems,",0.006


In [21]:
helpers.show_model(lda_model, corpus, dictionary)

Generating visual, this will take a few moments...


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.446768  0.009418       1        1  84.542297
4     -0.022582  0.167905       2        1   3.482391
5      0.042346 -0.087548       3        1   3.086651
3     -0.043488 -0.016092       4        1   2.625241
12    -0.029964 -0.022031       5        1   1.572702
7     -0.039839 -0.010991       6        1   1.114667
8     -0.027960 -0.013384       7        1   0.901683
11    -0.037615 -0.009640       8        1   0.729298
6     -0.044029 -0.006602       9        1   0.597988
10    -0.044709 -0.005079      10        1   0.514862
0     -0.041176 -0.005843      11        1   0.353066
14    -0.041035  0.002764      12        1   0.231599
13    -0.039393 -0.000771      13        1   0.115523
1     -0.039399 -0.001104      14        1   0.108340
9     -0.037926 -0.001003      15        1   0.023686, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
244   Default   85.000000         French   85.000000  30.0000  30.0000
13    Default   79.000000       computer   79.000000  29.0000  29.0000
1     Default  621.000000            and  621.000000  28.0000  28.0000
0     Default   49.000000       Computer   49.000000  27.0000  27.0000
5     Default  679.000000             of  679.000000  26.0000  26.0000
6     Default   43.000000        science   43.000000  25.0000  25.0000
180   Default   19.000000             de   19.000000  24.0000  24.0000
4     Default  230.000000             is  230.000000  23.0000  23.0000
328   Default   24.000000           more   24.000000  22.0000  22.0000
38    Default   57.000000           such   57.000000  21.0000  21.0000
77    Default   13.000000       language   13.000000  20.0000  20.0000
150   Default   48.000000          first   48.000000  19.0000  19.0000
288   Default   12.000000           only   12.000000  18.0000  18.0000
222   Default   35.000000           than   35.000000  17.0000  17.0000
347   Default  112.000000           that  112.000000  16.0000  16.0000
132   Default  111.000000            for  111.000000  15.0000  15.0000
356   Default   16.000000     University   16.000000  14.0000  14.0000
90    Default   19.000000      computers   19.000000  13.0000  13.0000
12    Default   18.000000  computational   18.000000  12.0000  12.0000
193   Default   31.000000           used   31.000000  11.0000  11.0000
216   Default    9.000000            had    9.000000  10.0000  10.0000
194   Default    9.000000           when    9.000000   9.0000   9.0000
30    Default  518.000000             in  518.000000   8.0000   8.0000
240   Default  160.000000            was  160.000000   7.0000   7.0000
2408  Default   10.000000         Middle   10.000000   6.0000   6.0000
234   Default   63.000000             it   63.000000   5.0000   5.0000
218   Default   22.000000           many   22.000000   4.0000   4.0000
656   Default   12.000000           data   12.000000   3.0000   3.0000
26    Default   17.000000      computing   17.000000   2.0000   2.0000
18    Default   20.000000       software   20.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
4370  Topic15    0.000661    L'Angelier.    0.391961   1.9625  -8.3827
1458  Topic15    0.000661        Boehm's    0.391883   1.9627  -8.3827
1452  Topic15    0.000661            set    0.393003   1.9598  -8.3827
1453  Topic15    0.000661  such-and-such    0.392364   1.9614  -8.3827
1454  Topic15    0.000661          that;    0.392364   1.9614  -8.3827
1455  Topic15    0.000661           then    0.393187   1.9593  -8.3827
1456  Topic15    0.000661          this.    0.391865   1.9627  -8.3827
1457  Topic15    0.000661           ways    0.393210   1.9593  -8.3827
1459  Topic15    0.000661           Note    0.391883   1.9627  -8.3827
1501  Topic15    0.000661      operates.    0.391890   1.

# Conclusions

By now, you have experimented with two preprocessing functions and saw their impact on topic modeling. Spend a few minutes reflecting on what you learned from your experience and put a few notes in the **Topic Modeling Conclusions** slide.

# References

1. https://dl.acm.org/doi/fullHtml/10.1145/2133806.2133826
2. https://nlpforhackers.io/topic-modeling/
3. https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/  -- for more analyses of topic models